# Juice Transfer Example
This example shows the trajectory of the JUpiter ICy moons Explorer (JUICE). The data for the planets and JUICE are retrieved from JPL Horizons via the astroquery library. The example showcases all the most commonly required features available in GUPTA.

#### Imports

In [1]:
from trajectoryanimator.animator import (
    TrajectoryAnimator,
    TrajectoryParticle,
    CameraSequence,
    astronomical_unit,
)
import numpy as np

from astroquery.jplhorizons import Horizons
from astropy import units
import datetime


#### Data retrieval
Most of the work for this example is in the data retrieval. For this animation we would like to moving the planets a bit ahead of the launch but starting JUICE at the launch. We setup the the start and end dates, as well as the astroquery epoch dictionaries below:

In [2]:
start = datetime.datetime(year=2022, month=1, day=1)
launch_juice = datetime.datetime(year=2023, month=4, day=15)
end = datetime.datetime(year=2031, month=7, day=21)

date_str_format = "%Y-%m-%d"

epochs_dict = epochs = {
    "start": start.strftime(date_str_format),
    "stop": end.strftime(date_str_format),
    "step": "5d",
}
epochs_dict_juice = epochs = {
    "start": launch_juice.strftime(date_str_format),
    "stop": end.strftime(date_str_format),
    "step": "5d",
}

The exact workings of astroquery and its JPL horizons interface are not important to this example, and will thus be left as an exercise to the reader. GUPTA requires a position array of shape 3xN and a list of datetimes of length N. We can use the following function to process the data retrieved by astroquery:

In [3]:
def retrieve_pos_and_times(horizons_query):
    vectors = horizons_query.vectors()

    position_data = np.vstack(
        [
            vectors["x"].to(units.meter),
            vectors["y"].to(units.meter),
            vectors["z"].to(units.meter),
        ]
    ).T
    time_data = list(vectors["datetime_str"])
    horizons_time_format = "A.D. %Y-%b-%d %H:%M:%S.%f"
    time_data = [datetime.datetime.strptime(x, horizons_time_format) for x in time_data]

    return time_data, position_data


Lets now retrieve the required data from astroquery:

In [4]:
juice_query = Horizons(id="JUICE", location="@Sun", epochs=epochs_dict_juice)
venus_query = Horizons(id="299", location="@Sun", epochs=epochs_dict)
earth_query = Horizons(id="399", location="@Sun", epochs=epochs_dict)
mars_query = Horizons(id="499", location="@Sun", epochs=epochs_dict)
jupiter_query = Horizons(id="599", location="@Sun", epochs=epochs_dict)

juice_times, juice_position = retrieve_pos_and_times(juice_query)
earth_times, earth_position = retrieve_pos_and_times(earth_query)
mars_times, mars_position = retrieve_pos_and_times(mars_query)
jupiter_times, jupiter_position = retrieve_pos_and_times(jupiter_query)
venus_times, venus_position = retrieve_pos_and_times(venus_query)

#### Creating a basic animation
Now that the data is ready, we can create a basic animation. GUPTA requires 2 object classes to create an animation. The `TrajectoryParticle` object contains the information of a single celestial body. The `TrajectoryAnimator` object collects all the information of the animation and is ultimately used to generate the trajectory. We give each Particle a name and colour as identification. Any [matplotlib-compatible colour](https://matplotlib.org/stable/gallery/color/named_colors.html) will work.

In [5]:
juice = TrajectoryParticle(
    name="JUICE",
    color="#FFD166",
    time_data=juice_times,
    position_data=juice_position,
)

earth = TrajectoryParticle(
    name="Earth",
    color="#249DAB",
    time_data=earth_times,
    position_data=earth_position,
)
venus = TrajectoryParticle(
    name="Venus",
    color="violet",
    time_data=venus_times,
    position_data=venus_position,
)
mars = TrajectoryParticle(
    name="Mars",
    color="#ef476f",
    time_data=mars_times,
    position_data=mars_position,
)
jupiter = TrajectoryParticle(
    name="Jupiter",
    color="#06D6A0",
    time_data=jupiter_times,
    position_data=jupiter_position,
)


In [ ]:
traj = TrajectoryAnimator(
    particles=[venus, earth, mars, jupiter, juice],
    speed=int(5e5),
    duration=None,
    central_body_color="yellow",
    plot_limits=5.5*astronomical_unit,
    fps=30,
    dpi=48,
    resolution=(1280, 720),
)


In [7]:
traj.run_animation(savefile="doc/juice_basic.gif")


Saving to doc/juice_basic.gif
Physical Duration: 3485 days, 0:00:00 | Animation Duration 0:00:20


  0%|          | 0/605 [00:00<?, ?it/s]

100%|█████████▉| 603/605 [01:20<00:00, 11.19it/s]

![Animation](doc/juice_basic.gif)

#### Additional improvements
Each of the trajectory particles allows for an `extra_data` which allows for displaying additional information about the particle. Lets add a T+- launch days. The extradata must be of shape NxM, where M is the number of extra_data entries. additionally an extra_data_prefixes list must be present with length M.

In [8]:
days_to_launch = [
    (
        str((x - launch_juice).days) + " days"
        if (x - launch_juice).days < 0
        else ("+" + str((x - launch_juice).days)) + " days"
    )
    for x in earth_times
]
days_to_launch = np.array(days_to_launch)[:, np.newaxis]

extra_data_prefixes = ["JUICE Launch T"]

print(days_to_launch[:5])
print(days_to_launch[-5:])

[['-469 days']
 ['-464 days']
 ['-459 days']
 ['-454 days']
 ['-449 days']]
[['+2996 days']
 ['+3001 days']
 ['+3006 days']
 ['+3011 days']
 ['+3016 days']]


To improve the readability of the animation, lets modify the lines, making JUICE's line thicker and adding a tracer to each of the bodies. We will also add a custom viewing angle using the camera object.

In [9]:

juice = TrajectoryParticle(
    "JUICE",
    "#FFD166",
    time_data=juice_times,
    position_data=juice_position,
    enable_tracer=False,
    # line_width=2,
)

earth = TrajectoryParticle(
    "Earth",
    "#249DAB",
    time_data=earth_times,
    position_data=earth_position,
    enable_tracer=True,
    extra_data=days_to_launch,
    extra_data_prefixes=extra_data_prefixes
)
venus = TrajectoryParticle(
    "Venus",
    "violet",
    time_data=venus_times,
    position_data=venus_position,
    enable_tracer=True,
)
mars = TrajectoryParticle(
    "Mars",
    "#ef476f",
    time_data=mars_times,
    position_data=mars_position,
    enable_tracer=True,
)
jupiter = TrajectoryParticle(
    "Jupiter",
    "#06D6A0",
    time_data=jupiter_times,
    position_data=jupiter_position,
    enable_tracer=True,
)

GUPTA's viewing angles can be adjusted with the `CameraSequence` class. The the camera can be setup with the `addSegment` method which acts as keyframes at specific points throughout the animation. These keyframes mark the viewing angle and zoom. The time of the keyframe is specified as a fraction, starting at 0 and ending at 1 when all particles have completed their orbit. Fractions beyond 1 allow for extension of the animation beyond the animation. Fractions below 0 are currently not implemented. The keyframes are smoothly animated using bezier curves. Lets set up a simple camera sequence:

In [10]:
camera = CameraSequence()
zoom0 = 1.2
zoom1 = 4
zoom2 = 2.5

camera.addSegment(0.0, elevation=90, azimuth=0, zoom = zoom0)
camera.addSegment(0.07, elevation=90, azimuth=0, zoom = zoom0)
camera.addSegment(0.10, elevation=90, azimuth=0, zoom = zoom1)
camera.addSegment(0.45, elevation=90, azimuth=0, zoom = zoom1)
camera.addSegment(0.6, elevation=90, azimuth=0, zoom = zoom2)
camera.addSegment(1, elevation=90, azimuth=0, zoom = zoom2)
camera.addSegment(1.1, elevation=90, azimuth=0, zoom = zoom2)

Lets use these changes to animate our final animation in both gif and mp4 format for the GitHub readme file. Please note that the mp4 will have a higher fidelity, so its use is generally preffered.

In [ ]:
traj = TrajectoryAnimator(
    particles=[venus, earth, mars, jupiter, juice],
    speed=int(3.5e5),
    central_body_color="yellow",
    camera=camera,
    plot_limits=5.5*astronomical_unit,
    fps=60,
    dpi=96,
    resolution=(1280, 720),
    watermark="Data retrieved from JPL Horizons system"
)

traj.run_animation(savefile="doc/juice_cam.gif")

![Animation](doc/juice_cam.gif)

Thats it! More documentation can be found in each function's docstrings. Try modifying the script to view the trajectory of other spacecraft or celestial bodies.